In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [103]:
# hyperparameters
batch_size = 1
num_examples = 5
d_in = 8
d_out = 4

In [104]:
# Our input example
X = torch.randn([batch_size,num_examples,d_in])


In [105]:
X

tensor([[[-1.8023,  1.3586, -0.2543, -0.2553, -0.6454,  0.7406,  0.0163,
           0.6733],
         [ 0.1661,  0.0428,  2.6721, -0.2022,  0.5432,  0.9540, -0.0680,
           1.4866],
         [ 0.0049,  1.0914,  0.5357,  1.5779,  0.4256, -0.1693,  0.9343,
          -0.3938],
         [-0.0477, -1.4761, -0.3724, -2.7631, -0.4528,  0.8164,  0.3976,
           0.4988],
         [ 0.1362, -1.4812, -0.5030,  0.8695,  1.0143, -0.6060,  1.6648,
          -0.1551]]])

In [ ]:
class selfattention(nn.Module):
    def __init__(self,d_in,d_out,device):
       super(selfattention,self).__init__()
       self.d_in = d_in
       self.d_out = d_out
       # Linear has better initialization compared to other ways of implementation I know
       self.device = device
       self.query = nn.Linear(d_in,d_out,device=device) ## d_in,d_out
       self.key = nn.Linear(d_in,d_out,device = device) ## d_in ,d_out
       self.value = nn.Linear(d_in,d_out, device = device) ## d_in,d_out
       self.softmax = nn.Softmax(dim = 1 )
    def forward(self,X):
        qx = self.query(X)
        kx = self.key(X)
        vx = self.value(X)
        # K.T
        kx = torch.transpose(kx,1,2)
        y = qx @ kx
        y /= (self.d_out)**0.5
        # creating a mask for causal attention
        mask = torch.ones(y.shape, device=self.device)
        mask = torch.triu(mask,diagonal=1)
        mask = mask.bool()
        # applying mask
        y  = torch.masked_fill(y,mask,-torch.inf)
        # applying softmax
        y = self.softmax(y)
        # y = (QK.T)@ V
        y = y @ qx  
        return y

In [107]:
device = "cpu"
attention_head = selfattention(d_in,d_out,device)


In [108]:
context_vector = attention_head(X)

In [109]:
print(context_vector)

tensor([[[-0.1069,  0.1238, -0.1181, -0.0028],
         [ 0.0635,  0.1817,  0.1037, -0.2806],
         [ 0.1578,  0.1338,  0.1367, -0.3292],
         [ 0.0933,  0.3457, -0.0870, -0.2294],
         [ 0.8883,  0.5244, -0.3898,  0.0803]]], grad_fn=<UnsafeViewBackward0>)
